# HW03: แบบจำลองภาษา (Laguage model)

ในหัวข้อนี้นักศึกษาจะได้สร้างแบบจำลองภาษา (N-gram model) โดยใช้ kenlm และเรียกใช้งานในรูปแบบต่างๆ

### 1) สร้างแบบจำลองภาษาด้วย Kenlm

ติดตั้ง kenlm ก่อน

In [36]:
!git clone https://github.com/kpu/kenlm.git

fatal: destination path 'kenlm' already exists and is not an empty directory.


In [37]:
%cd /content/kenlm
!pwd
!mkdir -p build
%cd build
!cmake ..
!make -j 4

%cd ../..

/content/kenlm
/content/kenlm
/content/kenlm/build
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Build files have been written to: /content/kenlm/build
[ 38%] Built target kenlm_util
[ 41%] Built target probing_hash_table_benchmark
[ 46%] Built target kenlm_filter
[ 71%] Built target kenlm
[ 73%] Built target query
[ 76%] Built target fragment
[ 78%] Built target build_binary
[ 81%] Built target kenlm_benchmark
[ 83%] Built target phrase_table_vocab
[ 86%] Built target filter
[ 95%] Built target kenlm_builder
[ 97%] Built target lmplz
[100%] Built target count_ngrams
/content


### ทำการสร้าง LM


ในตัวส่วนนี้จะมีข้อมูลอยู่ 2 ชุด
* ข้อมูลบทกลอนต่างๆ ของสุนทรภู่ ในตัวอย่างนี้เราจะใช้ข้อมูล จาก https://github.com/ThAIKeras/char-rnn-text-generation

* ข้อมูล Caption โดนๆ ที่ อ. รวบรวมมาจากเว็บไซต์ต่างๆ


In [38]:
!pip install pythainlp

In [39]:
#Prepare text for training LM
!wget https://raw.githubusercontent.com/ThAIKeras/char-rnn-text-generation/master/data/phra_aphai-train.txt

--2023-12-01 10:42:34--  https://raw.githubusercontent.com/ThAIKeras/char-rnn-text-generation/master/data/phra_aphai-train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5887331 (5.6M) [text/plain]
Saving to: ‘phra_aphai-train.txt.1’

phra_aphai-train.tx 100%[===================>]   5.61M  --.-KB/s    in 0.09s   

2023-12-01 10:42:34 (61.0 MB/s) - ‘phra_aphai-train.txt.1’ saved [5887331/5887331]



In [ ]:
from pythainlp.tokenize import word_tokenize

with open("segmented_text.txt","w",encoding='utf-8') as fw:
  with open("phra_aphai-train.txt",encoding='utf-8') as fp:
    lines = fp.readlines()

    for line in lines:
      words = word_tokenize(line, engine="newmm")
      fw.write(" ".join(words))

#/content/kenlm/build/bin/lmplz

In [ ]:
!head -n 5 segmented_text.txt

๏   แต่ ปาง หลัง ยังมี กรุง กษัตริย์ 
สมมุติ วงศ์ ทรง นาม ท้าว สุทัศน์   ผ่าน สมบัติ รัตนา นาม ธานี 
อัน กรุง ไกร ใหญ่ ยาว สิบ เก้า โยชน์   ภูเขา โขด เป็น กำแพง บุรี ศรี 
สะพรึบพร้อม ไพร่ฟ้า ประชาชี   ชาว บุรี หรรษา สถาวร 
มี เอก องค์ นงลักษณ์ อัคร ราช   พระนาง นาฏ นาม ปทุม เกสร 


In [ ]:
#-o 4 = 4-grams

!kenlm/build/bin/lmplz -o 4 < segmented_text.txt > sonthonphu_lm.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/segmented_text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 482509 types 13382
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:160584 2:1853694464 3:3475676928 4:5561083392
Statistics:
1 13382 D1=0.523536 D2=0.967964 D3+=1.52824
2 202618 D1=0.746751 D2=1.14058 D3+=1.45268
3 360557 D1=0.880307 D2=1.26298 D3+=1.58205
4 397739 D1=0.926755 D2=1.33805 D3+=1.47543
Memory estimate for binary LM:
type       kB
probing 20530 assuming -p 1.5
probing 23882 assuming -r models -p 1.5
trie     9098 without quantization
trie     4755 assuming -q 8 -b 8 quantization 
trie     8297 assuming -a 22 array pointer compression
trie     3954 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilitie

In [ ]:
!head -n 100 sonthonphu_lm.arpa

\data\
ngram 1=13382
ngram 2=202618
ngram 3=360557
ngram 4=397739

\1-grams:
-5.239251	<unk>	0
0	<s>	-1.1612302
-1.67113	</s>	0
-5.090665	๏	-0.12682416
-2.5159428	แต่	-0.3226692
-4.141145	ปาง	-0.25653675
-3.247343	หลัง	-0.2947313
-4.141145	ยังมี	-0.12682416
-3.3039086	กรุง	-0.6374317
-3.1807425	กษัตริย์	-0.37222502
-4.964257	สมมุติ	-0.12682416
-3.3894892	วงศ์	-0.42489177
-2.8088646	ทรง	-0.41256192
-3.5311425	นาม	-0.26183307
-3.006431	ท้าว	-0.5059465
-4.964257	สุทัศน์	-0.12682411
-3.5384862	ผ่าน	-0.41272008
-3.8497093	สมบัติ	-0.20829357
-4.141145	รัตนา	-0.28818932
-3.5771916	ธานี	-0.42094633
-3.0823011	อัน	-0.18709761
-4.0601864	ไกร	-0.22389036
-3.1906028	ใหญ่	-0.30076733
-3.6287088	ยาว	-0.22697361
-3.7310317	สิบ	-0.32466447
-4.0362387	เก้า	-0.21586354
-4.423601	โยชน์	-0.14589593
-4.0135427	ภูเขา	-0.19956611
-4.0135427	โขด	-0.18661165
-2.221873	เป็น	-0.44025958
-3.719644	กำแพง	-0.2522887
-3.4895113	บุรี	-0.27526405
-3.3306046	ศรี	-0.44374162
-5.090665	สะพรึบพร้อม	-0.12682416
-3.9330359	

In [ ]:
!tail -n 20 sonthonphu_lm.arpa

-0.9510361	เข้า ตี บ้าน เจ็ก
-0.7224537	พยายาม คง ก่อ ความไม่รู้
-0.89027417	ตัด ต้น จะ โค่น
-0.7694632	ถือ ลูก ขลุบ ตามยาว
-0.72600806	ป รา รม ภ๊
-1.0764453	อาย พวก ผี ขึ้
-0.7281465	ทั้ง เนื้อสด ไก่ ด้ม
-0.8684542	คลื่น ตลบ กระทบ ด้ง
-0.83981556	ไหว้ วอน พระเป็นเจ้า นั่งคุกเข่า
-0.7227579	บ่น พลาง คราง ขรม
-0.70851415	ลงมา ดวง ประไพ วิไลวรรณ
-0.6274052	ไม่ มีลูก เต ัา
-0.73385704	ทั้ง มั่งคั่ง ตั้ง ห้าง
-0.73254234	อิศ โร ดั่ง โกสีย์
-1.1094211	รบ นับ แสน แน่นขนัด
-0.7214427	จะ กำหนด กะ ระยะทาง
-0.71570617	จะ ดู เข็ม เต็มทน
-0.9337312	นิ่ง อิง หมอน ถอนหายใจ

\end\


In [ ]:
!pip install -U gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [ ]:
#ให้นักศึกษาลองเขียนโค้ดเพิ่มเติมเพื่อสร้าง LM ของชุดข้อมูล "Caption โดนๆ"

!gdown 1450RXQCdhPvBStqMsXaDEMdgoDSSn2Kp

Downloading...
From: https://drive.google.com/uc?id=1450RXQCdhPvBStqMsXaDEMdgoDSSn2Kp
To: /content/thai_cap.txt
100% 101k/101k [00:00<00:00, 178MB/s]


In [ ]:
from os import replace
from pythainlp.tokenize import word_tokenize

with open("thaicap_segmented_text.txt","w",encoding='utf-8') as fw:
  with open("thai_cap.txt",encoding='utf-8') as fp:
    lines = fp.readlines()

    for line in lines:
      if line.strip()=="":
        continue
      if line.strip()[0]=="#":
        continue
      words = word_tokenize(line, engine="newmm")
      fw.write(" ".join(words).replace(" "," ").replace(" "," "))

#/content/kenlm/build/bin/lmplz

In [ ]:
!kenlm/build/bin/lmplz -o 3 < thaicap_segmented_text.txt > thaicap_lm.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/thaicap_segmented_text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 8966 types 1721
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:20652 2:3788032768 3:7102561792
Statistics:
1 1721 D1=0.656642 D2=1.16602 D3+=1.05272
2 5776 D1=0.840509 D2=1.35373 D3+=1.23631
3 6808 D1=0.621849 D2=1.77321 D3+=2.57206
Memory estimate for binary LM:
type     kB
probing 298 assuming -p 1.5
probing 339 assuming -r models -p 1.5
trie    136 without quantization
trie     87 assuming -q 8 -b 8 quantization 
trie    132 assuming -a 22 array pointer compression
trie     83 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:20652 2:92416 3:136160
----5---10---15---20---25---3

### ประยุกต์ใช้งาน LM

จริงๆ แล้วก็มีหลายแอพพลิเคชันที่เราสามารถนำ LM ไปใช้งานได้ เช่น การตัดคำ การแปลภาษา การตรวจคำผิด การทำนายคำถัดไป โดยเราจะใช้ Python module ตัวนี้ในการอ่าน arpa file
https://pypi.org/project/arpa/

1) หาความน่าจะเป็นของการเกิดขึ้นของแต่ละประโยค

In [ ]:
!pip install arpa

In [ ]:
import arpa  # Python 3.4+

models = arpa.loadf("/content/sonthonphu_lm.arpa")
lm = models[0]  # ARPA files may contain several models.
print(len(models))

1


In [ ]:
print('p("ขม | ผัก ต้ม")',lm.p("ผัก ต้ม ขม"))

p("ขม | ผัก ต้ม") 0.13457227673544514


In [ ]:
print('p("หวาน | ผัก ต้ม")',lm.p("ผัก ต้ม หวาน"))

p("หวาน | ผัก ต้ม") 0.00016002174242816058


In [ ]:
# probability p(ขม, ผัก ต้ม)
print('p("ขม | ผัก ต้ม")',lm.p("ผัก ต้ม ขม"))
print('p("หวาน | ผัก ต้ม")',lm.p("ผัก ต้ม หวาน"))

print('log_p("ผัก ต้ม ขม")',lm.log_p("ผัก ต้ม ขม"))
print('log_p("ผัก ต้ม หวาน")',lm.log_p("ผัก ต้ม หวาน"))

# sentence score w/ sentence markers หมายถึงใส่ <s> ผัก ต้ม ขม ก็ ว่า หวาน </s>
#lm.s("ผัก ต้ม ขม ก็ ว่า หวาน")
print('log_s("ผัก ต้ม ขม ก็ ว่า หวาน")',lm.log_s("ผัก ต้ม ขม ก็ ว่า หวาน"))
print('log_s("ผัก ต้ม ขม ก็ ว่า ขม")',lm.log_s("ผัก ต้ม ขม ก็ ว่า ขม"))

# sentence score w/o sentence markers แบบไม่ใส่  <s> </s>
#lm.s("ผัก ต้ม ขม ก็ ว่า หวาน", sos=False, eos=False)
#lm.log_s("ผัก ต้ม ขม ก็ ว่า หวาน", sos=False, eos=False)

p("ขม | ผัก ต้ม") 0.13457227673544514
p("หวาน | ผัก ต้ม") 0.00016002174242816058
log_p("ผัก ต้ม ขม") -0.8710444
log_p("ผัก ต้ม หวาน") -3.795821005
log_s("ผัก ต้ม ขม ก็ ว่า หวาน") -17.192907525
log_s("ผัก ต้ม ขม ก็ ว่า ขม") -18.270590205


2) ลองทำนายคำถัดไป

ในส่วนนี้เราจะได้ประยุกต์ใช้งานในการ

In [ ]:
print(lm.vocabulary())

['</s>', '<s>', '<unk>', 'ก', 'กก', 'กกกอด', 'กค', 'กง', 'กงจักร', 'กงวาน', 'กงเกวียน', 'กงเกวียนกำเกวียน', 'กจน', 'กฎ', 'กฎหมาย', 'กด', 'กดคอ', 'กตัญญู', 'กติกา', 'กถา', 'กน', 'กนก', 'กนิษฐา', 'กนิษฐ์', 'กบ', 'กบฏ', 'กบิล', 'กบิลพัสดุ์', 'กปิตัน', 'กภิเษก', 'กม', 'กมล', 'กร', 'กรก', 'กรกฎ', 'กรง', 'กรงขัง', 'กรณ์', 'กรด', 'กรน', 'กรบ', 'กรม', 'กรมกรอม', 'กรมการ', 'กรมท่า', 'กรมนา', 'กรมวัง', 'กรมเกรียม', 'กรมเมือง', 'กรร', 'กรรชิง', 'กรรณ', 'กรรณิการ์', 'กรรม', 'กรรมฐาน', 'กรรมสิทธิ์', 'กรรมเวร', 'กรรม์', 'กรรเจียก', 'กรรเจียกจอน', 'กรวดทราย', 'กรวดน้ำ', 'กรวดน้ำคว่ำขัน', 'กรวบ', 'กรวม', 'กรวย', 'กรอ', 'กรอก', 'กรอง', 'กรองกรอย', 'กรองทอง', 'กรอด', 'กรอบ', 'กรอบหน้า', 'กรอบเกรียบ', 'กรอม', 'กระ', 'กระกูล', 'กระจก', 'กระจกส่อง', 'กระจง', 'กระจัง', 'กระจัด', 'กระจัดกระจาย', 'กระจัดพลัดพราย', 'กระจับ', 'กระจับปี่', 'กระจาด', 'กระจาย', 'กระจิบ', 'กระจี้', 'กระจุย', 'กระจ่าง', 'กระจ่างแจ้ง', 'กระจ้อยร่อย', 'กระฉอก', 'กระฉ่อน', 'กระชดกระช้อย', 'กระชับ', 'กระชั้น', 'กระชั้นชิด', 'กระชาก', 'ก

In [ ]:
print(list(lm._entries(2))[:10])
print(list(lm._entries(3))[:10])

[(-0.8423625, ('หลัง', '</s>'), 0), (-1.6904317, ('กษัตริย์', '</s>'), 0), (-1.3176723, ('วงศ์', '</s>'), 0), (-1.6275238, ('ทรง', '</s>'), 0), (-1.4049747, ('นาม', '</s>'), 0), (-2.1367214, ('ท้าว', '</s>'), 0), (-1.7816668, ('สมบัติ', '</s>'), 0), (-1.3912921, ('รัตนา', '</s>'), 0), (-0.5938672, ('ธานี', '</s>'), 0), (-1.840518, ('อัน', '</s>'), 0)]
[(-0.71986055, ('แต่', 'หลัง', '</s>'), 0), (-0.6972462, ('ปาง', 'หลัง', '</s>'), 0), (-0.60863173, ('สาม', 'หลัง', '</s>'), 0), (-0.60863173, ('คำ', 'หลัง', '</s>'), 0), (-0.60863173, ('ทั้งสอง', 'หลัง', '</s>'), 0), (-0.87188536, ('ลูบ', 'หลัง', '</s>'), 0), (-0.90673864, ('บน', 'หลัง', '</s>'), 0), (-0.60863173, ('หลาย', 'หลัง', '</s>'), 0), (-0.7295561, ('เห็น', 'หลัง', '</s>'), 0), (-0.9034992, ('อยู่', 'หลัง', '</s>'), 0)]


In [ ]:
print(lm.vocabulary())

print(list(lm._entries(2))[:10])
print(list(lm._entries(3))[:10])

#what is the next word
current_word = ['แต่', 'นั้น']

trigram = []
#หา 3-gram ว่ามี "แต่ นั้น xxx" ไหม
for item in list(lm._entries(3)):
  w1,w2,w3  = item[1] # (-0.6086742, ('สาม', 'หลัง', '</s>'), 0)
  #
  max_prob = None
  max_prob_sen = []
  if current_word[-2] == w1 and current_word[-1]== w2:
    print(item)

    _new_prob = lm.log_s(" ".join(current_word)+" "+w3)
    print(_new_prob)

['</s>', '<s>', '<unk>', 'ก', 'กก', 'กกกอด', 'กค', 'กง', 'กงจักร', 'กงวาน', 'กงเกวียน', 'กงเกวียนกำเกวียน', 'กจน', 'กฎ', 'กฎหมาย', 'กด', 'กดคอ', 'กตัญญู', 'กติกา', 'กถา', 'กน', 'กนก', 'กนิษฐา', 'กนิษฐ์', 'กบ', 'กบฏ', 'กบิล', 'กบิลพัสดุ์', 'กปิตัน', 'กภิเษก', 'กม', 'กมล', 'กร', 'กรก', 'กรกฎ', 'กรง', 'กรงขัง', 'กรณ์', 'กรด', 'กรน', 'กรบ', 'กรม', 'กรมกรอม', 'กรมการ', 'กรมท่า', 'กรมนา', 'กรมวัง', 'กรมเกรียม', 'กรมเมือง', 'กรร', 'กรรชิง', 'กรรณ', 'กรรณิการ์', 'กรรม', 'กรรมฐาน', 'กรรมสิทธิ์', 'กรรมเวร', 'กรรม์', 'กรรเจียก', 'กรรเจียกจอน', 'กรวดทราย', 'กรวดน้ำ', 'กรวดน้ำคว่ำขัน', 'กรวบ', 'กรวม', 'กรวย', 'กรอ', 'กรอก', 'กรอง', 'กรองกรอย', 'กรองทอง', 'กรอด', 'กรอบ', 'กรอบหน้า', 'กรอบเกรียบ', 'กรอม', 'กระ', 'กระกูล', 'กระจก', 'กระจกส่อง', 'กระจง', 'กระจัง', 'กระจัด', 'กระจัดกระจาย', 'กระจัดพลัดพราย', 'กระจับ', 'กระจับปี่', 'กระจาด', 'กระจาย', 'กระจิบ', 'กระจี้', 'กระจุย', 'กระจ่าง', 'กระจ่างแจ้ง', 'กระจ้อยร่อย', 'กระฉอก', 'กระฉ่อน', 'กระชดกระช้อย', 'กระชับ', 'กระชั้น', 'กระชั้นชิด', 'กระชาก', 'ก

In [ ]:

current_word = ['แต่', 'นั้น']
N = 5

trigram = []
#หา 3-gram ว่ามี "แต่ นั้น xxx" ไหม
for i in range(N):
  max_prob = None
  max_prob_next_word =""
  for item in list(lm._entries(3)):
    w1,w2,w3  = item[1] # (-0.6086742, ('สาม', 'หลัง', '</s>'), 0)


    if current_word[-2] == w1 and current_word[-1]== w2:
      #print(item[1][2])

      _new_prob = lm.log_s(" ".join(current_word)+" "+w3)
      #print(_new_prob)
      if max_prob==None or _new_prob > max_prob:
        max_prob = _new_prob
        max_prob_next_word = item[1][2]

print('max_prob',max_prob,max_prob_next_word)
current_word.append(max_prob_next_word)

max_prob -6.11323881 มา


In [ ]:
print(current_word)

['แต่', 'นั้น', 'มา']


จากตัวอย่างด้านบน เราก็จะรู้ว่าเราจะเลือก "แต่ นั้น มา" เพราะมีค่า prob มากกว่า นั่นเอง

3) เติมคำในช่องว่างด้วย LM

ในกรณีเราอยากแทรกคำที่หายไป ก็สามารถทำได้ เช่น

**เขาย่อมเปรียบเทียบความว่ายามรัก แต่น้ำ____ต้มขมชมว่าหวาน**

ถ้าให้ LM เดาจะได้คำว่าอะไรมาเติม อะไรประมาณนี้

In [ ]:
btext = "เขาย่อมเปรียบเทียบความว่ายามรักแต่น้ำ#ต้มขมชมว่าหวาน"
words = word_tokenize(btext, engine="newmm")
words
sent = " ".join(words).replace("#","[MASK]")

possible_words = []
for item in list(lm._entries(2)):
  w1,w2  = item[1] # (-0.6086742, ( 'หลัง', '</s>'), 0)
  if "น้ำ" == w1: # น้ำ xxx
    possible_words.append(w2)
  if "ต้ม" ==w2: # xxx ต้ม
    possible_words.append(w2)

print(sent)
print(set(possible_words))

N = 5
#หาความน่าจะเป็นของแต่ละประโยคถ้าเราแทนที่ [MASK] ด้วย possible_words
#แสดง
candidate_list = []
for pw in list(set(possible_words)):
  _new_text = sent.replace("[MASK]",pw)
  _new_prob = lm.log_s(_new_text)
  #print(_new_prob,_new_text)

  candidate_list.append([_new_prob,_new_text])

s = sorted(candidate_list, key = lambda x: (x[0]),reverse=True)

#มันมีคำต่อจากแม้น มากมาย เราจะเอาตัวที่ prob เยอะ 5 อันดับแรก จริงๆ วิธีนี้ไม่ค่อยถูกเท่าไหร่
for item in s[:N]:
  print(item)

#print(candidate_list)
#YOUR CODE HERE!!!!



นอกจากนี้จะมีอีกกรณีหนึ่ง เช่น เราต้องการสร้างรูปแบบประโยค สามารถมารถเกิดขึ้นได้ทั้งหมดแล้วหาว่า อันไหนมีค่า prob สูงสุด

In [ ]:
#สร้างประโยคจำนวน 8 คำ โดยจะเริ่มจากคำว่า แม้น

startWord = "แม้น"
numWord = 7
print(list(lm._entries(2))[:10])

sen_prob = []

#เราต้องวนหารูปแบบประโยคที่เป็นไปได้ทั้งหมด
for item in list(lm._entries(2)):
  w1,w2  = item[1] # (-0.6086742, ( 'หลัง', '</s>'), 0)
  #

  if startWord == w1:
    #print(item)
    _prob = lm.log_s(startWord + " "+ w2, eos=False)
    sen_prob.append([_prob, startWord + " "+ w2])


s = sorted(sen_prob, key = lambda x: (x[0]))

#มันมีคำต่อจากแม้น มากมาย เราจะเอาตัวที่ prob เยอะ 5 อันดับแรก จริงๆ วิธีนี้ไม่ค่อยถูกเท่าไหร่
for item in s[:5]:
  print(item)


#YOUR CODE HERE

## Assignment

* ให้นักศึกษาลองสร้าง Caption จากโมเดลภาษา โดยใช้ตัวอย่างโค้ดด้านบน จำนวนอย่างน้อย 5 แคปชัน

In [68]:
models = arpa.loadf("/content/thaicap_lm.arpa")
lm = models[0]  # ARPA files may contain several models.
print(len(models))

1


In [70]:
#what is the next word
current_word = ['โลก', 'นี้', 'ที่']
N = 12

trigram = []
for i in range(N):
  max_prob = None
  max_prob_next_word = ""
  for item in list(lm._entries(3)):
    w1,w2,w3  = item[1] # (-0.6086742, ('สาม', 'หลัง', '</s>'), 0)

    if current_word[-2] == w1 and current_word[-1]== w2:
      #print(item[1][2])
      _new_prob = lm.log_s(" ".join(current_word)+" "+w3)
      #print(_new_prob)
      if max_prob==None or _new_prob > max_prob:
        max_prob = _new_prob
        max_prob_next_word = item[1][2]

  print('max_prob',max_prob, max_prob_next_word)
  current_word.append(max_prob_next_word)

print(current_word)

max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
['โลก', 'นี้', 'ที่', '', '', '', '', '', '', '', '', '', '', '', '']


In [55]:
#what is the next word
current_word = ['ไม่', 'เป็น','อย่าง']
N = 12

trigram = []
for i in range(N):
  max_prob = None
  max_prob_next_word = ""
  for item in list(lm._entries(3)):
    w1,w2,w3  = item[1] # (-0.6086742, ('สาม', 'หลัง', '</s>'), 0)

    if current_word[-2] == w1 and current_word[-1]== w2:
      #print(item[1][2])
      _new_prob = lm.log_s(" ".join(current_word)+" "+w3)
      #print(_new_prob)
      if max_prob==None or _new_prob > max_prob:
        max_prob = _new_prob
        max_prob_next_word = item[1][2]

  print('max_prob',max_prob, max_prob_next_word)
  current_word.append(max_prob_next_word)

print(current_word)

max_prob -10.126053490999999 เอก
max_prob -11.289778171 อวด
max_prob -10.890990911 ผู้หญิง
max_prob -12.812883471 หยิ่ง
max_prob -11.832494126 ใจหาย
max_prob -13.503624126 </s>
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
['ไม่', 'เป็น', 'อย่าง', 'เอก', 'อวด', 'ผู้หญิง', 'หยิ่ง', 'ใจหาย', '</s>', '', '', '', '', '', '']


In [62]:
#what is the next word
current_word = ['เธอ', 'ที่', 'ที่']
N = 12

trigram = []
for i in range(N):
  max_prob = None
  max_prob_next_word = ""
  for item in list(lm._entries(3)):
    w1,w2,w3  = item[1] # (-0.6086742, ('สาม', 'หลัง', '</s>'), 0)

    if current_word[-2] == w1 and current_word[-1]== w2:
      #print(item[1][2])
      _new_prob = lm.log_s(" ".join(current_word)+" "+w3)
      #print(_new_prob)
      if max_prob==None or _new_prob > max_prob:
        max_prob = _new_prob
        max_prob_next_word = item[1][2]

  print('max_prob',max_prob, max_prob_next_word)
  current_word.append(max_prob_next_word)

print(current_word)

max_prob -11.29817657 พ่อ
max_prob -11.428146635000001 มา
max_prob -12.603230495 ราวี
max_prob -13.117180897659999 ฯ
max_prob -14.788310897659999 </s>
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
['เธอ', 'ที่', 'ที่', 'พ่อ', 'มา', 'ราวี', 'ฯ', '</s>', '', '', '', '', '', '', '']


In [63]:
#what is the next word
current_word = ['ใส่', 'นอน', 'ที่']
N = 12

trigram = []
for i in range(N):
  max_prob = None
  max_prob_next_word = ""
  for item in list(lm._entries(3)):
    w1,w2,w3  = item[1] # (-0.6086742, ('สาม', 'หลัง', '</s>'), 0)

    if current_word[-2] == w1 and current_word[-1]== w2:
      #print(item[1][2])
      _new_prob = lm.log_s(" ".join(current_word)+" "+w3)
      #print(_new_prob)
      if max_prob==None or _new_prob > max_prob:
        max_prob = _new_prob
        max_prob_next_word = item[1][2]

  print('max_prob',max_prob, max_prob_next_word)
  current_word.append(max_prob_next_word)

print(current_word)

max_prob -10.571946380000002 เตียง
max_prob -12.243076380000002 </s>
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
max_prob None 
['ใส่', 'นอน', 'ที่', 'เตียง', '</s>', '', '', '', '', '', '', '', '', '', '']


In [72]:
#what is the next word
current_word = ['เพราะ', 'ไหน', 'ที่']
N = 12

trigram = []
for i in range(N):
  max_prob = None
  max_prob_next_word = ""
  for item in list(lm._entries(3)):
    w1,w2,w3  = item[1] # (-0.6086742, ('สาม', 'หลัง', '</s>'), 0)

    if current_word[-2] == w1 and current_word[-1]== w2:
      #print(item[1][2])
      _new_prob = lm.log_s(" ".join(current_word)+" "+w3)
      #print(_new_prob)
      if max_prob==None or _new_prob > max_prob:
        max_prob = _new_prob
        max_prob_next_word = item[1][2]

  print('max_prob',max_prob, max_prob_next_word)
  current_word.append(max_prob_next_word)

print(current_word)

max_prob -9.025835476000001 มึง
max_prob -9.480698030000001 ได้
max_prob -10.124828540000001 เห็น
max_prob -10.578050540000001 คือ
max_prob -10.95175612 บท
max_prob -11.465169470000001 ที่
max_prob -11.757563556000001 มึง
max_prob -12.212426110000001 ได้
max_prob -12.856556620000001 เห็น
max_prob -13.309778620000001 คือ
max_prob -13.6834842 บท
max_prob -14.196897550000001 ที่
['เพราะ', 'ไหน', 'ที่', 'มึง', 'ได้', 'เห็น', 'คือ', 'บท', 'ที่', 'มึง', 'ได้', 'เห็น', 'คือ', 'บท', 'ที่']
